# Train Model on Baron - Human Dataset

In this notebook, we're going to train our model using the Baron Human dataset. 

This assumes that you've made the graph using the ```Infer GRN.ipynb``` code.

In [1]:
import os

import numpy as np
import pandas as pd
import torch
import torch_geometric
from torch_geometric.data import Data, Dataset
from tqdm import tqdm
from datasets.datasetbaronmouse import BaronMouseDataset
%matplotlib inline
import matplotlib.pyplot as plt
from sklearn.manifold import TSNE
from tqdm import tqdm
from sklearn.metrics import roc_auc_score, precision_recall_curve, auc

Torch version: 1.8.0+cu111
Cuda available: True
Torch geometric version: 2.0.3


In [2]:
dataset = BaronMouseDataset("/gpfs/data/rsingh47/hzaki1/data-baron-mouse")

100%|██████████| 139607/139607 [00:07<00:00, 19928.72it/s]


In [3]:
print()
print(f'Dataset: {dataset}:')
print('====================')
print(f'Number of graphs: {len(dataset)}')
print(f'Number of features: {dataset.num_features}')

data = dataset[0]  # Get the first graph object.

print()
print(data)
print('=============================================================')

# Gather some statistics about the first graph.
print(f'Number of nodes: {data.num_nodes}')
print(f'Number of edges: {data.num_edges}')
print(f'Average node degree: {data.num_edges / data.num_nodes:.2f}')
print(f'Contains isolated nodes: {data.contains_isolated_nodes()}')
print(f'Contains self-loops: {data.contains_self_loops()}')
print(f'Is undirected: {data.is_undirected()}')


Dataset: BaronMouseDataset(1886):
Number of graphs: 1886
Number of features: 1

Data(x=[14861], edge_index=[2, 139607], y=[1])
Number of nodes: 14861
Number of edges: 139607
Average node degree: 9.39
Contains isolated nodes: True
Contains self-loops: False
Is undirected: False


/gpfs/home/hzaki1/celltypefromgrn/env-gpu/lib/python3.7/site-packages/torch_geometric/deprecation.py:13: UserWarning: 'contains_isolated_nodes' is deprecated, use 'has_isolated_nodes' instead
  warnings.warn(out)
/gpfs/home/hzaki1/celltypefromgrn/env-gpu/lib/python3.7/site-packages/torch_geometric/deprecation.py:13: UserWarning: 'contains_self_loops' is deprecated, use 'has_self_loops' instead
  warnings.warn(out)


In [4]:
shuffle_index = np.loadtxt('shuffle_indices/shuffle_index_BaronMouse.txt')
shuffle_index = shuffle_index.astype(np.int32)
train_size, val_size = int(len(shuffle_index)* 0.8), int(len(shuffle_index)* 0.9)
train_data = [dataset[i] for i in shuffle_index[0:train_size]]
val_data = [dataset[i] for i in shuffle_index[train_size: val_size]]
test_data =  [dataset[i] for i in shuffle_index[val_size:]]

In [5]:
# torch.manual_seed(12345)
# dataset = dataset.shuffle()

# #train_dataset = torch.load('trainDataset.pt')
# #test_dataset = torch.load('testDataset.pt')

# train_dataset = dataset[:6856]
# test_dataset = dataset[6856:]
# #test_dataset = dataset[-12:]


# # train_dataset = dataset[:40]
# # test_dataset = dataset[40:60]

# print(f'Number of training graphs: {len(train_dataset)}')
# print(f'Number of test graphs: {len(test_dataset)}')

In [6]:
from torch_geometric.data import DataLoader

train_loader = DataLoader(train_data, batch_size=30, shuffle=True)
val_loader = DataLoader(val_data, batch_size=30, shuffle=True)
test_loader = DataLoader(test_data, batch_size=30, shuffle=False)

/gpfs/home/hzaki1/celltypefromgrn/env-gpu/lib/python3.7/site-packages/torch_geometric/deprecation.py:13: UserWarning: 'data.DataLoader' is deprecated, use 'loader.DataLoader' instead
  warnings.warn(out)


In [7]:
from gcnmodel import GCN
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = GCN(hidden_channels=128, data=dataset, output_size=13).to(device)
print(model)

GCN(
  (conv1): SAGEConv(1, 128)
  (conv2): SAGEConv(128, 128)
  (conv3): SAGEConv(128, 128)
  (lin): Linear(in_features=128, out_features=13, bias=True)
)


In [8]:
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
criterion = torch.nn.CrossEntropyLoss()

def train():
    model.train()
    avgLoss = 0
    for data in tqdm(train_loader, total=51):  # Iterate in batches over the training dataset.
        data.x = torch.reshape(data.x, (data.x.shape[0], 1))
        data.x = data.x.type(torch.FloatTensor)
        data = data.to(device)
        out = model(data.x, data.edge_index, data.batch)# Perform a single forward pass.
        loss = criterion(out, data.y)  # Compute the loss.
        loss.backward()  # Derive gradients.
        optimizer.step()  # Update parameters based on gradients.
        optimizer.zero_grad()  # Clear gradients.
        avgLoss += loss
    return avgLoss / 229

def test(loader):
    model.eval()

    correct = 0
    avgAUC = 0
    for data in loader:  # Iterate in batches over the training/test dataset.
        data.x = torch.reshape(data.x, (data.x.shape[0], 1))
        data.x = data.x.type(torch.FloatTensor)
        data = data.to(device)
        out = model(data.x, data.edge_index, data.batch)  
        pred = out.argmax(dim=1)  # Use the class with highest probability.
        correct += int((pred == data.y).sum())  # Check against ground-truth labels.
        avgAUCbatch = 0
        aucCounter = 0
        one_hot = np.eye(13)[data.y.cpu()]
        for i in range(0,13):
            try:
                roc = roc_auc_score(one_hot[:,i], out.cpu().detach().numpy()[:,i])
                avgAUCbatch += roc
                aucCounter +=1
            except Exception as e:
                continue
        if aucCounter == 0:
            continue
        avgAUC += avgAUCbatch/aucCounter
    return correct / len(loader.dataset), avgAUC / len(loader)  # Derive ratio of correct predictions.


for epoch in range(1, 250):
    loss = train()
    train_acc, trainAUC = test(train_loader)
    test_acc,testAUC = test(test_loader)
    print(f'Epoch: {epoch:03d}, Train Acc: {train_acc:.4f}, Train AUC: {trainAUC:.4f}, Test Acc: {test_acc:.4f}, Test Auc: {testAUC:.4f}, Loss: {loss:.4f}')

100%|██████████| 51/51 [00:04<00:00, 12.31it/s]


Epoch: 001, Train Acc: 0.6127, Train AUC: 0.6289, Test Acc: 0.5714, Test Auc: 0.6261, Loss: 1.5445


100%|██████████| 51/51 [00:03<00:00, 13.64it/s]


Epoch: 002, Train Acc: 0.5889, Train AUC: 0.7719, Test Acc: 0.5450, Test Auc: 0.7664, Loss: 0.3728


100%|██████████| 51/51 [00:03<00:00, 13.51it/s]


...

Epoch: 247, Train Acc: 0.9775, Train AUC: 0.9116, Test Acc: 0.7090, Test Auc: 0.8440, Loss: 0.0455


100%|██████████| 51/51 [00:03<00:00, 13.57it/s]


Epoch: 248, Train Acc: 0.9874, Train AUC: 0.8948, Test Acc: 0.6984, Test Auc: 0.8364, Loss: 0.0520


100%|██████████| 51/51 [00:03<00:00, 13.58it/s]


Epoch: 249, Train Acc: 0.9728, Train AUC: 0.9084, Test Acc: 0.6825, Test Auc: 0.8439, Loss: 0.0462


In [9]:
torch.save(model.state_dict(), 'model_weights/model_weightsBaronMouse.pth')

In [12]:
dataset.cellToIndex

{'beta': 0,
 'ductal': 1,
 'delta': 2,
 'schwann': 3,
 'quiescent_stellate': 4,
 'endothelial': 5,
 'gamma': 6,
 'alpha': 7,
 'macrophage': 8,
 'immune_other': 9,
 'activated_stellate': 10,
 'B_cell': 11,
 'T_cell': 12}

In [23]:
torch.save(train_dataset, 'train_loader_aug24.pt')
torch.save(test_dataset, 'test_loader_aug24.pt')

In [8]:
model.load_state_dict(torch.load('model_weightsSept20BaronHuman.pth'))

<All keys matched successfully>

In [11]:
from sklearn.metrics import average_precision_score

def testForAUPR(loader):
    model.eval()

    correct = 0
    avgAUC = 0
    toReturn = np.zeros((len(loader), 14))
    ind = 0
    actual = np.zeros((len(loader), 14))
    for data in loader:  # Iterate in batches over the training/test dataset.
        data.x = torch.reshape(data.x, (data.x.shape[0], 1))
        data.x = data.x.type(torch.FloatTensor)
        data = data.to(device)
        out = model(data.x, data.edge_index, data.batch)
        wowow = average_precision_score(data.y.cpu().detach().numpy(), out.cpu().detach().numpy(), average='micro')
        print(wowowow)
    
testForAUPR(test_loader)

ValueError: multiclass format is not supported